# Shannon-Fano-Codierung

Die Codierung nach Shannon-Fano ist ein verlustloser Kompressionsalgorithmus. Sie gliedert die Symbole sukzessive in Teilmengen auf, bis jede Teilmenge nur noch ein Symbol enthält und eine eindeutige Bitfolge zugeordnet werden kann. Die Elemente werden zunächst entsprechend ihrer Auftrittswahrscheinlichkeiten sortiert. Sie werden dann in zwei Teilmengen aufgeteilt, so dass beide Teilmengen ungefähr die gleiche Auftrittswahrscheinlichkeit besitzen. Der oberen Teilmenge wird beispielsweise das Bit 0, der unteren das Bit 1 zugeordnet. Beide Teilmengen werden dann nach dem gleichen Prinzip weiter unterteilt, bis jedes Element eine eindeutige Bitfolge erhalten hat. Es entsteht wie bei der Huffman-Codierung ein binärer Baum, an dessen Verzweigungen die Bit 0 bzw. 1 zugeordnet werden. 

Die Shannon-Fano-Codierung generiert einen präfixfreien Code und ist damit verlustlos. Sie liefert nicht zwingend die bestmögliche Kompression, ist also suboptimal. Allerdings stimmen die Ergebnisse von Huffman- und Shannon-Fano-Codierung häufig überein.

In [1]:
# Python-Code zur Shannon-Fano-Codierung
# code adapted from https://rosettacode.org/wiki/Huffman_coding#Python

from operator import itemgetter
import numpy as np

Shannon_Fano_dict={}

def Shannon_Fano_coding(seq, code):
    a = {}
    b = {}
    if len(seq) == 1:
        Shannon_Fano_dict[seq.popitem()[0]] = code
        return 0
 
    prob_cum = 0
    prob_sum = sum(seq.values())
    for i in sorted(seq.items(), key=itemgetter(1), reverse=False):
        prob_cum += i[1]
        if prob_cum < prob_sum/2:
            a[i[0]] = seq[i[0]]
        elif (prob_sum-sum(a.values())) > (sum(a.values())+i[1]):
            a[i[0]] = seq[i[0]]
        else:
            b[i[0]] = seq[i[0]]
    Shannon_Fano_coding(a, code + "0")
    Shannon_Fano_coding(b, code + "1")

Im folgenden Abschnitt wird der zu komprimierende Text definiert und statistisch analysiert.

In [2]:
#zu komprimierender Text 
text = "Auch hier lohnt es sich wieder, auf den Unterschied zwischen den untersuchten Textsorten hinzuweisen, da sich die Hamburger Gruppe nicht allein auf wissenschaftliche Informationstexte beschränkt. Trotzdem sei an dieser Stelle festgehalten, dass informationsdichtere Texte, also Texte mit einer höheren semantischen Redundanz, wie sie der theoretisch-deduktive Ansatz empfiehlt, für gewöhnlich einen behaltensfördernden Effekt erzeugen und sich entsprechend positiv auf die Verständlichkeit eines Textes auswirken – aber auch die Lesezeit unter Umständen wesentlich erhöhen."
text_length_symbols = len(text)

print ("Folgender Text bestehend aus %g Zeichen soll nun komprimiert werden.\n\n %s \n" % (text_length_symbols,text))

# Generiere Liste mit Symbolen (Buchstaben) und ihren Auftrittswahrscheinlichkeiten
symbol_count = {}
for ch in text:
    if ch not in symbol_count:
        symbol_count[ch] = 1
    else:
        symbol_count[ch] += 1

# symbol_count = {"A": 25, "B": 23, "C": 22, "D": 10, "E": 8, "F": 7, "G": 5}


# Anzahl unterschiedlicher Zeichen in text
symbol_number = len(symbol_count)
print("\nDer Text enthält %g verschiedene Symbole.\n" % (symbol_number))

# Bestimme Auftrittswahrscheinlichkeit der einzelnen Symbole
prob = np.zeros(len(symbol_count))
cntr = 0
for sym, freq in symbol_count.items():
    prob[cntr] = freq / text_length_symbols
    cntr +=1



# Berechne Entropie des Textes
entropy = - np.inner(prob, np.log2(prob))

print("Die Entropie des Alphabetes ist %1.2f bit, die des Textes %1.2f bit.\n" % (entropy, entropy*text_length_symbols) )

Folgender Text bestehend aus 573 Zeichen soll nun komprimiert werden.

 Auch hier lohnt es sich wieder, auf den Unterschied zwischen den untersuchten Textsorten hinzuweisen, da sich die Hamburger Gruppe nicht allein auf wissenschaftliche Informationstexte beschränkt. Trotzdem sei an dieser Stelle festgehalten, dass informationsdichtere Texte, also Texte mit einer höheren semantischen Redundanz, wie sie der theoretisch-deduktive Ansatz empfiehlt, für gewöhnlich einen behaltensfördernden Effekt erzeugen und sich entsprechend positiv auf die Verständlichkeit eines Textes auswirken – aber auch die Lesezeit unter Umständen wesentlich erhöhen. 


Der Text enthält 42 verschiedene Symbole.

Die Entropie des Alphabetes ist 4.34 bit, die des Textes 2485.36 bit.



In [3]:
# einfache binäre Codierung ohne Berücksichtigung der Auftrittswahrscheinlichkeiten
equal_symbol_length_bit = np.ceil(np.log2(symbol_number))
equal_text_length_bit = equal_symbol_length_bit * text_length_symbols

print("Bei einer einfachen binären Codierung ohne Berücksichtigung der Auftrittswahrscheinlichkeiten müssten %g bit pro Zeichen aufgewendet werden.\n" % (equal_symbol_length_bit))
print("Der codierte Text hätte eine Länge von %g bit.\n" %(equal_text_length_bit))

Bei einer einfachen binären Codierung ohne Berücksichtigung der Auftrittswahrscheinlichkeiten müssten 6 bit pro Zeichen aufgewendet werden.

Der codierte Text hätte eine Länge von 3438 bit.



Mit dem obigen Python-Code kann nun die Shannon-Fano-Codierung durchgeführt werden. Die binären Codeworte sind von links nach rechts zu lesen.

In [4]:
# Rufe Routine für Shannon-Fano-Codierung auf
Shannon_Fano_coding(symbol_count, "")   # dictionary

# sortierte Liste des Dictionaries
sorted_symbol_count = sorted(((value, key) for (key,value) in symbol_count.items()), reverse=True)

#print(Shannon_Fano_dict.values())
print("Symbol\tGewicht\tShannon-Fano-Code")
code_len = np.zeros(len(symbol_count))
cntr = 0
for p in sorted_symbol_count:
    print("%s\t%s\t%s" % (p[1], p[0], Shannon_Fano_dict[p[1]]))
    code_len[cntr] = len(Shannon_Fano_dict[p[1]])
    cntr +=1


Symbol	Gewicht	Shannon-Fano-Code
e	88	111
 	71	110
n	45	101
i	40	1001
t	38	1000
s	36	0111
h	31	0110
r	28	0101
d	23	01001
a	20	01000
c	19	00111
u	16	00110
l	13	001011
f	12	001010
o	10	001001
w	8	000111
m	8	001000
z	7	0001101
,	6	0001100
x	5	000100
p	5	0001010
k	5	0001011
T	5	0000111
ö	4	0000110
g	4	0000101
b	4	00001001
ä	3	00001000
v	2	000001111
U	2	00000110
A	2	00000101
.	2	000001110
–	1	000001000
ü	1	000000110
V	1	0000001111
S	1	000000011
R	1	000000100
L	1	000001001
I	1	000000010
H	1	000000000
G	1	000000001
E	1	0000001110
-	1	000000101


In [5]:
# Berechnung der Entropie des Textes (Annahme: Text ist repräsentativ.)
fano_symbol_length_bit = np.inner(prob,code_len)
fano_text_length_bit = fano_symbol_length_bit*text_length_symbols
print("Die mittlere Wortlänge des Shannon-Fano-Codes beträgt %1.2f bit pro Symbol.\n" % (fano_symbol_length_bit))
print("Die Länge des codierten Textes ist %g Bit.\n" % (fano_text_length_bit))

Die mittlere Wortlänge des Shannon-Fano-Codes beträgt 4.96 bit pro Symbol.

Die Länge des codierten Textes ist 2843 Bit.



In [6]:
# Bestimmung der Redundanz des Shannon-Fano-Codes
redundancy = fano_symbol_length_bit - entropy
print("Die Redundanz des Shannon-Fano-Codes beträgt für dieses Beispiel %1.2f bit oder %1.2f%%.\n" % (redundancy,redundancy/fano_symbol_length_bit*100))
print("Verglichen mit der Codierung ohne Kompression konnte die Datenmenge um den Faktor %1.2f reduziert werden.\n" %(equal_text_length_bit/fano_text_length_bit))

Die Redundanz des Shannon-Fano-Codes beträgt für dieses Beispiel 0.62 bit oder 12.58%.

Verglichen mit der Codierung ohne Kompression konnte die Datenmenge um den Faktor 1.21 reduziert werden.

